In [2]:
import numpy as np
from scipy.stats import norm, t
import math
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib
from tqdm import tqdm


In [ ]:
def Parisian_Call(K,L,n):
    # number of simulation 
    N = 250 # number of time steps
    r = 0.05 # risk-free rate
    sigma = 0.25 # Volatility
    D = 0.1 # least peiod
    T = 1 # total time
    S_0 = 100 # initial price
    #n = 100000 # number of simulation 
    #  # of simulation 

  
    res = []
    #price_paths = []
    for i in tqdm(range(n)):

        
        # simulated prices:
        prices = [0] * (N+1)
        # initial price is S_0
        prices[0] = S_0
        # Discretize time step
        activation = 0

        dt = T/N # time period
        maxcount = 0
        excess_count = 0
        for t_i in range(1,N+1,1):
            # counting the period excess the barrier
            
            # time interval
            Z = np.random.normal(0, np.sqrt(dt))
            prices[t_i] = prices[t_i-1]* np.exp((r-0.5*sigma**2)*dt+sigma*Z)
            
            
            if prices[t_i-1] < L and prices[t_i]>= L:
                #excess_count = 0
                excess_count += 0.5*dt 
            
            elif prices[t_i-1] >= L and prices[t_i] >= L:
                excess_count += dt
                maxcount = max(maxcount,excess_count)
            elif prices[t_i-1] >= L and prices[t_i] < L:
                excess_count += 0.5*dt 
                maxcount = max(excess_count,maxcount)
                # check activation
                excess_count = 0

            else:
                pass

          
        if maxcount >= D:
            activation = 1
            
        res.append(max(0,activation*(prices[-1]-K))*np.exp(0-r*T))
        #price_paths.append(prices)


        
    #resarray = np.array(res)
    #res = resarray*np.exp(0-r*T)
    return res



In [ ]:
# K = 110, L = 120
res = Parisian_Call(K=110,L=120,n=100000)

mu_mc = np.mean(res)
print("MC estimate for mu is {}".format(mu_mc))
# unbaised standard deviation:
sum_of_square = 0
for realized in res:
    sum_of_square += (realized-mu_mc)**2
sample_std = np.sqrt(sum_of_square/(len(res)-1))

# Z-score for 95% confidence
z_score = norm.ppf(.975)

[mu_mc-z_score*sample_std/np.sqrt(len(res)),
        mu_mc+z_score*sample_std/np.sqrt(len(res))]
print(f"95% Confidence Interval: ({mu_mc-z_score*sample_std/np.sqrt(len(res))}, {mu_mc+z_score*sample_std/np.sqrt(len(res))})")

In [ ]:
# K = 120, L = 110
res = Parisian_Call(K=120,L=110,n=100000)

mu_mc = np.mean(res)
print("MC estimate for mu is {}".format(mu_mc))
# unbaised standard deviation:
sum_of_square = 0
for realized in res:
    sum_of_square += (realized-mu_mc)**2
sample_std = np.sqrt(sum_of_square/(len(res)-1))

# Z-score for 95% confidence
z_score = norm.ppf(.975)

[mu_mc-z_score*sample_std/np.sqrt(len(res)),
        mu_mc+z_score*sample_std/np.sqrt(len(res))]
print(f"95% Confidence Interval: ({mu_mc-z_score*sample_std/np.sqrt(len(res))}, {mu_mc+z_score*sample_std/np.sqrt(len(res))})")

In [ ]:
class ParisianCallOption:
    def __init__(self, K, L, N=250, r=0.05, sigma=0.25, D=0.1, T=1, S_0=100):
        self.K = K  # Strike price
        self.L = L  # Barrier level
        self.N = N  # Number of time steps
        self.r = r  # Risk-free rate
        self.sigma = sigma  # Volatility
        self.D = D  # Least period
        self.T = T  # Total time
        self.S_0 = S_0  # Initial price
        self.results = []  # To store simulation results
    
    def simulate(self, n):
        """
        Simulate n paths of the underlying asset price and calculate the payoff
        of the Parisian call option.
        """
        for i in tqdm(range(n)):
            prices = [self.S_0]
            dt = self.T / self.N
            maxcount = 0
            excess_count = 0
            for t_i in range(1, self.N+1):
                Z = np.random.normal(0, np.sqrt(dt))
                prices.append(prices[-1] * np.exp((self.r - 0.5 * self.sigma**2) * dt + self.sigma * Z))
                if prices[t_i-1] < self.L and prices[t_i] >= self.L:
                    excess_count += 0.5 * dt
                elif prices[t_i-1] >= self.L and prices[t_i] >= self.L:
                    excess_count += dt
                    maxcount = max(maxcount, excess_count)
                elif prices[t_i-1] >= self.L and prices[t_i] < self.L:
                    excess_count += 0.5 * dt
                    maxcount = max(excess_count, maxcount)
                    excess_count = 0

            activation = maxcount >= self.D
            self.results.append(max(0, activation * (prices[-1] - self.K)) * np.exp(-self.r * self.T))
    
    def calculate_metrics(self):
        """
        Calculate mean, standard deviation, and 95% confidence interval of the option price.
        """
        mu_mc = np.mean(self.results)
        sample_std = np.sqrt(np.sum((self.results - mu_mc)**2) / (len(self.results) - 1))
        z_score = norm.ppf(0.975)
        ci_lower = mu_mc - z_score * sample_std / np.sqrt(len(self.results))
        ci_upper = mu_mc + z_score * sample_std / np.sqrt(len(self.results))
        return mu_mc, (ci_lower, ci_upper)
    
    def run_simulation(self, n):
        """
        Run simulation and print out the results.
        """
        self.simulate(n)
        mu_mc, confidence_interval = self.calculate_metrics()
        print(f"MC estimate for mu is {mu_mc}")
        print(f"95% Confidence Interval: {confidence_interval}")

# Example usage:
option1 = ParisianCallOption(K=110, L=120)
option1.run_simulation(100000)  # Adjust n for the number of simulations as needed

option2 = ParisianCallOption(K=120, L=110)
option2.run_simulation(100000)  # Adjust n for the number of simulations as needed